In [ ]:
"""
Version 1.0.0, Evaluate the visual results
"""

import torch
import clip
from PIL import Image
from typing import List, Dict
import requests
import matplotlib.pyplot as plt
import transformers
from transformers import CLIPProcessor, CLIPModel, AutoProcessor, CLIPVisionModel

def cal_text2image_similarity(text: List[str], image: Image, processor: CLIPProcessor, model: CLIPModel):
    image = Image.open(image)
    inputs = processor(text=text, images=image, return_tensors="pt", padding=True)
    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image # this is the image-text similarity score
    return logits_per_image

model = CLIPModel.from_pretrained("/zecheng/svg_model_hub/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("/zecheng/svg_model_hub/clip-vit-base-patch32")


In [ ]:
text = ["head"]
image = "/workspace/zecheng/SUWA/output.png"
res = cal_text2image_similarity(text, image, processor, model)
print(res)

In [13]:
import torch
import clip
from PIL import Image
from typing import List, Dict
import requests
import matplotlib.pyplot as plt
import transformers
from pytorch_fid import fid_score
from transformers import CLIPProcessor, CLIPModel, AutoProcessor, CLIPVisionModel
import numpy as np

model = CLIPVisionModel.from_pretrained("/zecheng/svg_model_hub/clip-vit-base-patch32")
processor = AutoProcessor.from_pretrained("/zecheng/svg_model_hub/clip-vit-base-patch32")

image1, image2 = "/workspace/zecheng/SUWA/output.png", "/workspace/zecheng/SUWA/output.png"

# def cal_fid_score(image1: Image, image2: Image, clip_model: CLIPModel, device: str):
#     """
#     Calculate the FID score

image1, image2 = Image.open(image1), Image.open(image2)

inputs1 = processor(images=image1, return_tensors="pt")
inputs2 = processor(images=image2, return_tensors="pt")

outputs1, outputs2 = model(**inputs1), model(**inputs2)
features1, features2 = outputs1.last_hidden_state, outputs2.last_hidden_state

# Calculate FID score  
# fid = fid_score.calculate_fid_given_paths([features1, features2], dims=2048, device='cuda', batch_size=features1.size(0)) 
from torch import FloatTensor

def __to_numpy_array(feats) -> np.ndarray:
    if isinstance(feats, list):
        # flatten list of batch-processed features
        if isinstance(feats[0], FloatTensor):
            feats = [x.detach().cpu().numpy() for x in feats]
    else:
        feats = feats.detach().cpu().numpy()
    return np.concatenate(feats)

features1 = __to_numpy_array(features1)
features2 = __to_numpy_array(features2)

m1 = features1.mean(axis=0)
s1 = np.cov(features1, rowvar=False)  
  
m2 = features2.mean(axis=0)  
s2 = np.cov(features2, rowvar=False)  

eps = 1e-6  
s1 += eps * np.eye(s1.shape[0])  
s2 += eps * np.eye(s2.shape[0]) 

fid_value = fid_score.calculate_frechet_distance(m1, s1, m2, s2)  


print(fid_value)

2.054302683518472e-08


In [10]:
outputs1, outputs2 = model(**inputs1), model(**inputs2)
features1, features2 = outputs1.last_hidden_state, outputs2.last_hidden_state

features1

features1 = __to_numpy_array(features1)

In [5]:
from torch import FloatTensor

def __to_numpy_array(feats) -> np.ndarray:
    if isinstance(feats, list):
        # flatten list of batch-processed features
        if isinstance(feats[0], FloatTensor):
            feats = [x.detach().cpu().numpy() for x in feats]
    else:
        feats = feats.detach().cpu().numpy()
    return np.concatenate(feats)

features1 = __to_numpy_array(features1)
features1 = __to_numpy_array(features1)

m1 = features1.mean(axis=0)
s1 = np.cov(features1, rowvar=False)  
  
m2 = features2.mean(axis=0)  
s2 = np.cov(features2, rowvar=False)  
  
fid_value = fid_score.calculate_frechet_distance(m1, s1, m2, s2)  


AttributeError: 'numpy.ndarray' object has no attribute 'detach'